![msg](msg.jpg)

In [ ]:
#Imports 
import astropy.units as u
import astropy.constants as const
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 

from tqdm import tqdm

from DiscEvolution.grid import Grid
from DiscEvolution.star import SimpleStar
from DiscEvolution.eos import IrradiatedEOS  
from DiscEvolution.disc import AccretionDisc
from DiscEvolution.viscous_evolution import ViscousEvolutionFV
from DiscEvolution.driver import PlanetDiscDriver
from DiscEvolution.io import Event_Controller

In [16]:
# Physical parameters
#Assumed
Mstar = 1.0 * u.solMass #Default 1 solmass 
Rstar = 1.0 * u.solRad #Default 1 solRad
mu = 2.33  # Mean molecular weight, assumed

#Given
M_disk_0 = 0.05 * u.solMass  #Initial Disk Mass, default is 0.05 solmass
gamma = 1.0
Rc = 1.0 * u.AU #Default 1AU
#Mdot_0 = 
t_final = 1 * u.Myr 

# LBP surface density profile with gamma = 1: Sigma(r) = (M_disk / (2*pi*Rc*r)) * exp(-r/Rc)
def Sigma_LBP_1(r) -> float:
    val = (M_disk_0)/(2* np.pi * Rc * r) * np.exp((-1*r)/(Rc))
    if (val.cgs.unit != (u.g / (u.cm)**2)):
        print("Sigma_1 error")
        return 0
    return val.cgs

def Temperature(r) -> float:
    T_0 = 150 * u.K 
    val  = T_0 * (r / (1.0 * u.AU))**(-1/2)
    if val.cgs.unit != u.K:
        print('Temperature error')
        return 0
    return val.cgs

#Taken from A2
def Omega(r) -> float:
    val = np.sqrt((const.G*Mstar)/(r**3))
    if (val.cgs.unit != (1 / u.s)): 
        print("Omega error")
        return 0
    return val.cgs

#Taken from A2
def c_s(r) -> float:
    val = np.sqrt((const.k_B*Temperature(r))/(mu*const.m_p))
    if (val.cgs.unit != (u.cm / u.s)): 
        print("c_s error")
        return 0
    return val.cgs

def H(r) -> float: 
    val = c_s(r)/Omega(r)
    if val.cgs.unit != (u.cm):
        print("H error")
        return 0
    return val.cgs

In [17]:
#Given Mdisk, Mdot(0), Rc -> Calculate alpha and Sigma0
M_disk_0 = 0.05 * u.solMass
Mdot_0 = 1E-7 * u.solMass / u.yr  #Armitage p.21
Rc = 1.0 * u.AU

#Armitage eq 74
#nu*Sigma = Mdot/(3pi) * (1 - sqrt(r_star/r))
##alpha * c_s * H * Sigma = (Mdot / 3*pi) * (1 - sqrt(r_star/r))
###alpha = (Mdot / 3*pi) / (c_s * H * Sigma) * (1 - sqrt(r_star/r))

def alpha(r) -> float:
    # Convert all inputs to CGS values (no units)
    Mdot_cgs = Mdot_0.to(u.g / u.s).value  # g/s
    cs_val = c_s(r).value                   # cm/s
    H_val = H(r).value                      # cm
    Sigma_val = Sigma_LBP_1(r).value       # g/cm²
    Rstar_val = Rstar.to(u.cm).value       # cm
    r_val = r.to(u.cm).value               # cm
    
    # Calculate dimensionless alpha
    alpha = (Mdot_cgs / (3*np.pi)) * (1/(cs_val * H_val * Sigma_val)) * (1 - np.sqrt(Rstar_val/r_val))
    
    return alpha  # Pure float, dimensionless

r_ref = Rc #Not Rstar
alpha_val = alpha(r_ref)

Sigma_0 = Sigma_LBP_1(r_ref)

print(f"Accretion rate: {Mdot_0:.3e}")
print(f"At radius: {r_ref.to(u.AU):.3f}")
print(f"\nCalculated alpha: {alpha_val:.3e}")
print(f"Sigma_0: {Sigma_0.cgs:.3e}")


Accretion rate: 1.000e-07 solMass / yr
At radius: 1.000 AU

Calculated alpha: 8.973e-04
Sigma_0: 2.601e+04 g / cm2
